In [4]:
import gc
import json
import pandas as pd
import numpy as np
import seaborn as sns

from tqdm import tqdm_notebook
from glob import glob

from sklearn.decomposition import TruncatedSVD, NMF
from utils import loadpkl, to_json

%matplotlib inline

In [15]:
# load feathers
files = sorted(glob('../features/*.feather'))
df = pd.concat([pd.read_feather(f) for f in tqdm_notebook(files, mininterval=60)], axis=1)

# split train & test
train_df = df[df['click_mode'].notnull()]
test_df = df[df['click_mode'].isnull()]

/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pandas/io/feather_format.py:124: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  nthreads=int_use_threads)
/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pyarrow/pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [17]:
# load pkls
df = loadpkl('../features/plans.pkl')
queries = loadpkl('../features/queries.pkl')
profiles = loadpkl('../features/profiles.pkl')
queries_pred = loadpkl('../features/queries_pred.pkl')

In [3]:
# merge
df = pd.merge(df, queries, on='sid', how='left')
df = pd.merge(df, profiles, on='pid', how='left')

del queries, profiles
gc.collect()

train_df = df[df['click_mode'].notnull()]

In [18]:
profiles = profiles[['pid']+['profile_{}'.format(i) for i in range(0,66)]]
df = pd.merge(queries, profiles, on='pid', how='left')

In [21]:
df.drop(['pid','req_time'],axis=1,inplace=True)

In [22]:
df

,sid,click_mode,x_o,y_o,x_d,y_d,queries_o_count,queries_d_count,queries_x_o_count,queries_y_o_count,...,profile_56,profile_57,profile_58,profile_59,profile_60,profile_61,profile_62,profile_63,profile_64,profile_65
0,3000821,9.0,116.29,39.97,116.32,39.96,1101,1046,15792,20412,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3085857,1.0,116.39,39.84,116.33,39.79,695,185,33319,12712,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2944522,9.0,116.31,39.93,116.27,40.00,635,4633,27853,33989,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,559931,1.0,116.27,39.88,116.39,39.90,363,8756,11881,15663,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2819352,7.0,116.34,39.96,116.37,39.86,1166,15545,18655,17994,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,2754575,1.0,116.54,39.73,116.48,39.76,55,75,3515,2952,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,2224795,1.0,116.34,39.73,116.79,40.35,312,12,18655,2952,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,1598541,1.0,116.26,40.22,116.24,40.22,350,745,5885,3308,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,676316,5.0,116.39,39.91,116.39,39.90,3473,8756,33319,61527,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1068802,3.0,116.29,40.02,116.27,39.95,114,513,15792,4817,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
